<h2><center><b><i>Cluster bomb</b></i>: Uncovering Patterns in Terrorist Group Beliefs and Attacks</center></h2>

#### **COM-480: Data Visualization**

**Team**: Alexander Sternfeld, Silvia Romanato & Antoine Bonnet

**Dataset**: [Global Terrorism Database (GTD)](https://www.start.umd.edu/gtd/) 

**Additional dataset**: [Profiles of Perpetrators of Terrorism in the United States (PPTUS)](https://dataverse.harvard.edu/dataset.xhtml?persistentId=hdl%3A1902.1/17702)

## Cluster analysis
 

In [14]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)

BASE_DIR = os.path.dirname(os.getcwd())
DATA_DIR = os.path.join(BASE_DIR, 'data')

### 1. Loading the data

In [15]:
# If the data is already combined, read it
GTD_path = os.path.join(DATA_DIR, 'GTD.pkl')

if os.path.exists(GTD_path):
    GTD = pd.read_pickle(GTD_path)
elif os.path.exists('data/GTD1.xlsx') and os.path.exists('data/GTD2.xlsx'):
    # Read data, ignore unicode errors
    GTD_1 = pd.read_excel('data/GTD1.xlsx')
    GTD_2 = pd.read_excel('data/GTD2.xlsx')
    # Append the two dataframes and save in a serialized format
    GTD = GTD_1.append(GTD_2)
    GTD.to_pickle(GTD_path)
else:
    print('GTD data not found, please download and put it in data folder.')

GTD.head()


,eventid,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,region,region_txt,provstate,city,latitude,longitude,specificity,vicinity,location,summary,crit1,crit2,crit3,doubtterr,alternative,alternative_txt,multiple,success,suicide,attacktype1,attacktype1_txt,attacktype2,attacktype2_txt,attacktype3,attacktype3_txt,targtype1,targtype1_txt,targsubtype1,targsubtype1_txt,corp1,target1,natlty1,natlty1_txt,targtype2,targtype2_txt,targsubtype2,targsubtype2_txt,corp2,target2,natlty2,natlty2_txt,targtype3,targtype3_txt,targsubtype3,targsubtype3_txt,corp3,target3,natlty3,natlty3_txt,gname,gsubname,gname2,gsubname2,gname3,gsubname3,motive,guncertain1,guncertain2,guncertain3,individual,nperps,nperpcap,claimed,claimmode,claimmode_txt,claim2,claimmode2,claimmode2_txt,claim3,claimmode3,claimmode3_txt,compclaim,weaptype1,weaptype1_txt,weapsubtype1,weapsubtype1_txt,weaptype2,weaptype2_txt,weapsubtype2,weapsubtype2_txt,weaptype3,weaptype3_txt,weapsubtype3,weapsubtype3_txt,weaptype4,weaptype4_txt,weapsubtype4,weapsubtype4_txt,weapdetail,nkill,nkillus,nkillter,nwound,nwoundus,nwoundte,property,propextent,propextent_txt,propvalue,propcomment,ishostkid,nhostkid,nhostkidus,nhours,ndays,divert,kidhijcountry,ransom,ransomamt,ransomamtus,ransompaid,ransompaidus,ransomnote,hostkidoutcome,hostkidoutcome_txt,nreleased,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
0,197000000001,1970,7,2,NaN,0,NaT,58,Dominican Republic,2,Central America & Caribbean,National,Santo Domingo,18.456792,-69.951164,1.0,0,NaN,NaN,1,1,1,0,NaN,NaN,0.0,1,0,1,Assassination,NaN,NaN,NaN,NaN,14,Private Citizens & Property,68.0,Named Civilian,NaN,Julio Guzman,58.0,Dominican Republic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MANO-D,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGIS,0,0,0,0,NaN
1,197000000002,1970,0,0,NaN,0,NaT,130,Mexico,1,North America,Federal,Mexico city,19.371887,-99.086624,1.0,0,NaN,NaN,1,1,1,0,NaN,NaN,0.0,1,0,6,Hostage Taking (Kidnapping),NaN,NaN,NaN,NaN,7,Government (Diplomatic),45.0,"Diplomatic Personnel (outside of embassy, cons...",Belgian Ambassador Daughter,"Nadine Chaval, daughter",21.0,Belgium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23rd of September Communist League,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,1.0,1.0,0.0,NaN,NaN,NaN,Mexico,1.0,800000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGIS,0,1,1,1,NaN
2,197001000001,1970,1,0,NaN,0,NaT,160,Philippines,5,Southeast Asia,Tarlac,Unknown,15.478598,120.599741,4.0,0,NaN,NaN,1,1,1,0,NaN,NaN,0.0,1,0,1,Assassination,NaN,NaN,NaN,NaN,10,Journalists & Media,54.0,Radio Journalist/Staff/Facility,Voice of America,Employee,217.0,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
3,197001000002,1970,1,0,NaN,0,NaT,78,Greece,8,Western Europe,Attica,Athens,37.997490,23.762728,1.0,0,NaN,NaN,1,1,1,0,NaN,NaN,0.0,1,0,3,Bombing/Explosion,NaN,NaN,NaN,NaN,7,Government (Diplomatic),46.0,Embassy/Consulate,NaN,U.S. Embassy,217.0,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,Explosives,16.0,Unknown Explosive Type,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Explosive,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,Na

In [16]:
# Read PPTUS dataset or create it 
PPTUS_path = os.path.join(DATA_DIR, 'PPTUS.pkl')
PPTUS_filepath = os.path.join(DATA_DIR, 'PPT-US_0517dist.xlsx')

if os.path.exists(PPTUS_path): 
    PPTUS = pd.read_pickle(PPTUS_path)
elif os.path.exists(PPTUS_filepath):
    PPTUS = pd.read_excel(PPTUS_filepath)
    PPTUS.to_pickle(PPTUS_path)
else:
    print('PPTUS data not found, please download and put it in data/ folder.')

PPTUS.head()

,ORGNAME,MOD_DATE,INGROUP,ALIAS,ALIAS_CI,ALIAS_SOURCE,GATK_FIRST_MONTH,GATK_FIRST_DAY,GATK_FIRST_YEAR,GATK_FIRST_CI,GATK_FIRST_SOURCE,GATK_LAST_MONTH,GATK_LAST_DAY,GATK_LAST_YEAR,GATK_LAST_CI,GATK_LAST_SOURCE,USATK_FIRST_MONTH,USATK_FIRST_DAY,USATK_FIRST_YEAR,USATK_FIRST_CI,USATK_FIRST_SOURCE,USATK_LAST_MONTH,USATK_LAST_DAY,USATK_LAST_YEAR,USATK_LAST_CI,USA_LAST_SOURCE,LOC_ATK_COUNTRY_1,LOC_ATK_COUNTRY_1_CI,LOC_ATK_COUNTRY_1_SOURCE,LOC_ATK_COUNTRY_2,LOC_ATK_COUNTRY_2_CI,LOC_ATK_COUNTRY_2_SOURCE,LOC_ATK_COUNTRY_3,LOC_ATK_COUNTRY_3_CI,LOC_ATK_COUNTRY_3_SOURCE,LOC_ATK_COUNTRY_4,LOC_ATK_COUNTRY_4_CI,LOC_ATK_COUNTRY_4_SOURCE,LOC_ATK_COUNTRY_5,LOC_ATK_COUNTRY_5_CI,LOC_ATK_COUNTRY_5_SOURCE,LOC_HQ_COUNTRY_1,LOC_HQ_COUNTRY_1_CI,LOC_HQ_COUNTRY_1_SOURCE,LOC_HQ_COUNTRY_2,LOC_HQ_COUNTRY_2_CI,LOC_HQ_COUNTRY_2_SOURCE,LOC_HQ_COUNTRY_3,LOC_HQ_COUNTRY_3_CI,LOC_HQ_COUNTRY_3_SOURCE,LOC_HQ_COUNTRY_4,LOC_HQ_COUNTRY_4_CI,LOC_HQ_COUNTRY_4_SOURCE,LOC_HQ_COUNTRY_5,LOC_HQ_COUNTRY_5_CI,LOC_HQ_COUNTRY_5_SOURCE,LOC_USATK_STATE_1,LOC_USATK_STATE_CI_1,LOC_USATK_STATE_SOURCE_1,LOC_USATK_STATE_2,LOC_USATK_STATE_CI_2,LOC_USATK_STATE_SOURCE_2,LOC_USATK_STATE_3,LOC_USATK_STATE_CI_3,LOC_USATK_STATE_SOURCE_3,LOC_USATK_STATE_4,LOC_USATK_STATE_CI_4,LOC_USATK_STATE_SOURCE_4,LOC_USATK_STATE_5,LOC_USATK_STATE_CI_5,LOC_USATK_STATE_SOURCE_5,LOC_USAHQ_STATE_1,LOC_USAHQ_STATE_CI_1,LOC_USAHQ_STATE_SOURCE_1,LOC_USAHQ_STATE_2,LOC_USAHQ_STATE_CI_2,LOC_USAHQ_STATE_SOURCE_2,LOC_USAHQ_STATE_3,LOC_USAHQ_STATE_CI_3,LOC_USAHQ_STATE_SOURCE_3,LOC_USAHQ_STATE_4,LOC_USAHQ_STATE_CI_4,LOC_USAHQ_STATE_SOURCE_4,LOC_USAHQ_STATE_5,LOC_USAHQ_STATE_CI_5,LOC_USAHQ_STATE_SOURCE_5,HIS,HIS_CLEAN,HIS_CI,HIS_SOURCE,DFORM_MONTH,DFORM_DAY,DFORM_YEAR,DFORM_CI,DFORM_SOURCE,PHIL,PHIL_CLEAN,PHIL_CI,PHIL_SOURCE,NOTABLE,NOTABLE _CLEAN,NOTABLE_CI,NOTABLE_SOURCE,DOM_I,DOM_I_CI,DOM_I_SOURCE,I_ETHNO,I_ETHNO_CI,I_ETHNO_SOURCE,I_REL,I_REL_CI,I_REL_SOURCE,I_REL_1,I_REL_1_CI,I_REL_1_SOURCE,I_REL_2,I_REL_2_CI,I_REL_2_SOURCE,I_REL_3,I_REL_3_CI,I_REL_3_SOURCE,I_REL_4,I_REL_4_CI,I_REL_4_SOURCE,I_REL_5,I_REL_5_CI,I_REL_5_SOURCE,I_REL_6,I_REL_6_CI,I_REL_6_SOURCE,I_REL_7,I_REL_7_CI,I_REL_7_SOURCE,I_REL_8,I_REL_8_CI,I_REL_8_SOURCE,I_REL_9,I_REL_9_CI,I_REL_9_SOURCE,I_REL_10,I_REL_10_CI,I_REL_10_SOURCE,I_RACE,I_RACE_CI,I_RACE_SOURCE,I_RACE_1,I_RACE_1_CI,I_RACE_1_SOURCE,I_RACE_2,I_RACE_2_CI,I_RACE_2_SOURCE,I_RACE_3,I_RACE_3_CI,I_RACE_3_SOURCE,I_LEFT,I_LEFT_CI,I_LEFT_SOURCE,I_LEFT_1,I_LEFT_1_CI,I_LEFT_1_SOURCE,I_LEFT_2,I_LEFT_2_CI,I_LEFT_2_SOURCE,I_LEFT_3,I_LEFT_3_CI,I_LEFT_3_SOURCE,I_LEFT_4,I_LEFT_4_CI,I_LEFT_4_SOURCE,I_LEFT_5,I_LEFT_5_CI,I_LEFT_5_SOURCE,I_LEFT_6,I_LEFT_6_CI,I_LEFT_6_SOURCE,I_RIGHT,I_RIGHT_CI,I_RIGHT_SOURCE,I_RIGHT_1,I_RIGHT_1_CI,I_RIGHT_1_SOURCE,I_RIGHT_2,I_RIGHT_2_CI,I_RIGHT_2_SOURCE,I_RIGHT_3,I_RIGHT_3_CI,I_RIGHT_3_SOURCE,I_RIGHT_4,I_RIGHT_4_CI,I_RIGHT_4_SOURCE,I_SI,I_SI_CI,I_SI_SOURCE,I_SI_1,I_SI_1_CI,I_SI_1_SOURCE,I_SI_2,I_SI_2_CI,I_SI_2_SOURCE,I_SI_3,I_SI_3_CI,I_SI_3_SOURCE,I_SI_4,I_SI_4_CI,I_SI_4_SOURCE,I_SI_5,I_SI_5_CI,I_SI_5_SOURCE,I_SI_6,I_SI_6_CI,I_SI_6_SOURCE,I_SI_7,I_SI_7_CI,I_SI_7_SOURCE,I_SI_8,I_SI_8_CI,I_SI_8_SOURCE,I_SI_9,I_SI_9_CI,I_SI_9_SOURCE,I_SI_10,I_SI_10_CI,I_SI_10_SOURCE,I_SI_11,I_SI_11_CI,I_SI_11_SOURCE,I_SI_12,I_SI_12_CI,I_SI_12_SOURCE,I_SI_13,I_SI_13_CI,I_SI_13_SOURCE,I_SI_14,I_SI_14_CI,I_SI_14_SOURCE,G_POL_1,G_POL_1_CI,G_POL_1_SOURCE,G_POL_2,G_POL_2_CI,G_POL_2_SOURCE,G_POL_3,G_POL_3_CI,G_POL_3_SOURCE,G_POL_4,G_POL_4_CI,G_POL_4_SOURCE,G_POL_5,G_POL_5_CI,G_POL_5_SOURCE,G_POL_OTH,G_POL_OTH_TXT,G_POL_OTH_CI,G_POL_OTH_SOURCE,G_SOC_1,G_SOC_1_CI,G_SOC_1_SOURCE,G_SOC_2,G_SOC_2_CI,G_SOC_2_SOURCE,G_SOC_OTH,G_SOC_OTH_TXT,G_SOC_OTH_CI,G_SOC_OTH_SOURCE,G_ECO_1,G_ECO_1_CI,G_ECO_1_SOURCE,G_ECO_2,G_ECO_2_CI,G_ECO_2_SOURCE,G_ECO_OTH,G_ECO_OTH_TXT,G_ECO_OTH_CI,G_ECO_OTH_SOURCE,G_REL_1,G_REL_1_CI,G_REL_1_SOURCE,G_REL_2,G_REL_2_CI,G_REL_2_SOURCE,G_REL_3,G_REL_3_CI,G_REL_3_SOURCE,G_REL_OTH,G_REL_OTH_TXT,G_REL_OTH_CI,G_REL_OTH_SOURCE,OA_POL_1,OA_POL_1_CI,OA_POL_1_SOURCE,OA_POL_2,OA_POL_2_CI,OA_POL_2_SOUR